In [1]:
import os
import numpy as np
import tensorflow as tf

In [2]:
IMG_SIZE = 160

def format_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    return image, label

train_dataset = tf.keras.utils.image_dataset_from_directory(
    "club-logo/data/train", label_mode = "categorical", image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=0.1, subset="training", shuffle=True,
    seed=42, batch_size=32
)

valid_dataset = tf.keras.utils.image_dataset_from_directory(
    "club-logo/data/train", label_mode = "categorical", image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=0.1, subset="validation", shuffle=True,
    seed=42, batch_size=32
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    "club-logo/data/test", label_mode = "categorical", image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True, seed=42, batch_size=32
)

train_dataset = train_dataset.map(format_example)
valid_dataset = valid_dataset.map(format_example)
test_dataset = test_dataset.map(format_example)



Found 3000 files belonging to 3 classes.
Using 2700 files for training.
Found 3000 files belonging to 3 classes.
Using 300 files for validation.
Found 960 files belonging to 3 classes.


2024-05-22 00:06:23.225143: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-05-22 00:06:23.225165: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-22 00:06:23.225172: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-22 00:06:23.225191: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-22 00:06:23.225202: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

In [4]:
base_model.trainable = False

In [5]:
next(iter(train_dataset.take(1)))

(<tf.Tensor: shape=(32, 160, 160, 3), dtype=float32, numpy=
 array([[[[-1., -1., -1.],
          [-1., -1., -1.],
          [-1., -1., -1.],
          ...,
          [-1., -1., -1.],
          [-1., -1., -1.],
          [-1., -1., -1.]],
 
         [[-1., -1., -1.],
          [-1., -1., -1.],
          [-1., -1., -1.],
          ...,
          [-1., -1., -1.],
          [-1., -1., -1.],
          [-1., -1., -1.]],
 
         [[-1., -1., -1.],
          [-1., -1., -1.],
          [-1., -1., -1.],
          ...,
          [-1., -1., -1.],
          [-1., -1., -1.],
          [-1., -1., -1.]],
 
         ...,
 
         [[-1., -1., -1.],
          [-1., -1., -1.],
          [-1., -1., -1.],
          ...,
          [-1., -1., -1.],
          [-1., -1., -1.],
          [-1., -1., -1.]],
 
         [[-1., -1., -1.],
          [-1., -1., -1.],
          [-1., -1., -1.],
          ...,
          [-1., -1., -1.],
          [-1., -1., -1.],
          [-1., -1., -1.]],
 
         [[-1., -1., -1.

In [6]:
image_batch,label_batch = next(iter(train_dataset.take(1)))
print(label_batch)
feature_batch = base_model(image_batch)
print(feature_batch.shape)
feature_batch

tf.Tensor(
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]], shape=(32, 3), dtype=float32)
(32, 5, 5, 1280)


<tf.Tensor: shape=(32, 5, 5, 1280), dtype=float32, numpy=
array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 3.4127264 , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 2.6455665 , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.94251585, ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]],

        [[0.        , 0.        , 1.1822824 , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 6.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 6.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 3.9735422 , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.     

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)
feature_batch_average

(32, 1280)


<tf.Tensor: shape=(32, 1280), dtype=float32, numpy=
array([[0.0000000e+00, 0.0000000e+00, 1.7090362e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 4.4483814e-02, 7.1996993e-01, ..., 0.0000000e+00,
        2.9650155e-01, 0.0000000e+00],
       [0.0000000e+00, 4.5610312e-02, 1.1545796e+00, ..., 0.0000000e+00,
        9.7976681e-03, 0.0000000e+00],
       ...,
       [1.8025634e-01, 0.0000000e+00, 3.4893316e-01, ..., 0.0000000e+00,
        0.0000000e+00, 8.7187022e-02],
       [5.7273157e-02, 2.8777465e-02, 1.8338048e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.8736622e-01, 1.2649810e+00, 2.1154010e-01, ..., 0.0000000e+00,
        9.0002059e-04, 0.0000000e+00]], dtype=float32)>

In [8]:
prediction_layer = tf.keras.layers.Dense(3)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


(32, 3)


In [9]:
x = base_model.output;
x = global_average_layer(x)
x = prediction_layer(x)


model = tf.keras.Model(inputs=base_model.input,outputs=x)

In [10]:
base_learning_rate = 0.0001

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
	loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
	metrics=['accuracy']
)

In [11]:
initial_epochs = 10 # please try `initial_epochs = 10` or `initial_epochs = 20` after `initial_epochs = 2` is okay
validation_steps=20

loss0,accuracy0 = model.evaluate(valid_dataset, steps = validation_steps)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

2024-05-22 00:06:24.941269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 0.2617 - loss: 1.3995 
initial loss: 1.37
initial accuracy: 0.27


2024-05-22 00:06:27.683393: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[ArgMax_1/_14]]
2024-05-22 00:06:27.683417: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-05-22 00:06:27.683441: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9055564564477267138
2024-05-22 00:06:27.683450: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 3709463722619234681
2024-05-22 00:06:27.683463: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 3635462544952403552
2024-05-22 00:06:27.683476: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16963483623833462342
2024-05-22 00:06

In [12]:
import keras

loss_result = []
accuracy_result = []

class PerBatchMetrics(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        loss_result.append(logs['loss'])
        accuracy_result.append(logs['accuracy'])



## The actual training step
history = model.fit(
    train_dataset,
    epochs=initial_epochs,
    validation_data=valid_dataset,
    callbacks=[PerBatchMetrics()]
)

Epoch 1/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - accuracy: 0.6099 - loss: 0.8701 - val_accuracy: 0.9933 - val_loss: 0.1980
Epoch 2/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.9980 - loss: 0.1530 - val_accuracy: 1.0000 - val_loss: 0.0397
Epoch 3/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.9995 - loss: 0.0349 - val_accuracy: 1.0000 - val_loss: 0.0104
Epoch 4/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 74ms/step - accuracy: 0.9998 - loss: 0.0098 - val_accuracy: 1.0000 - val_loss: 0.0035
Epoch 5/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.9996 - loss: 0.0042 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 6/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.9998 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 7.9004e-04
Epoch 7/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.9996 - loss: 0.0014 - val_accuracy: 1.0000 - val_loss: 5.1031e-04
Epoch 8/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.9997 - loss: 9.0109e-04 - val_accurac

In [15]:
model.save('my_model.keras', overwrite=True)